In [1]:
!pip3 install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  distutils: /private/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/pip-build-env-e4onap9x/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/pip-build-env-e4onap9x/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/pip-build-env-e4onap9x/normal'
  distutils: /private/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/pip-build-env-e4onap9x/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/pip-build-env-e4onap9x/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  r

In [2]:
import cv2
import face_recognition
import numpy as np
import speech_recognition as sr
import threading

/Users/rachelchen/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
rachel_image = face_recognition.load_image_file("rachel.jpg")
rachel_encoding = face_recognition.face_encodings(rachel_image)[0]

xander_image = face_recognition.load_image_file("xander.jpg")
xander_encoding = face_recognition.face_encodings(xander_image)[0]

james_image = face_recognition.load_image_file("james.jpg")
james_encoding = face_recognition.face_encodings(james_image)[0]

IndexError: list index out of range

In [ ]:
known_face_encodings = [
    rachel_encoding,
    xander_encoding
]
known_face_names = [
    "Rachel Chen",
    "Xander Chin"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


In [ ]:

video_capture = cv2.VideoCapture(1)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color to RGB color
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])

    # Find all the faces and face encodings in the frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # Check if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.5)
        name = "Unknown"

        # Use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        else:
            # If the face is not recognized, add it to the known faces list
            known_face_encodings.append(face_encoding)
            name = f"Unnamed Person"
            known_face_names.append(name)

        face_names.append(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [6]:
def audio_transcription():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something!")
        while True:
            audio = recognizer.listen(source)
            try:
                # Transcribe the audio to text
                text = recognizer.recognize_google(audio)
                print("Transcription: " + text)
            except sr.UnknownValueError:
                print("Could not understand audio")
            except sr.RequestError as e:
                print("Error; {0}".format(e))

In [7]:
# Running camera operations in a separate thread
camera_thread = threading.Thread(target=camera_operations)
camera_thread.start()

# Running audio transcription in the main thread
audio_transcription()

# Wait for the camera thread to finish
camera_thread.join()

AttributeError: Could not find PyAudio; check installation

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/q4/w5wd298x2j51mk7bkv38kmt80000gn/T/ipykernel_11648/1519530862.py", line 55, in camera_operations
cv2.error: Unknown C++ exception from OpenCV code


: 

In [14]:
from openai import OpenAI
client = OpenAI()

audio_file = open("speech.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable